In [20]:
 #* these libraries will be put in the requeriments.txt file, stay her for now
!pip install scikit-learn
!pip install tensorflow
!pip install opencv-python
!pip install pillow




[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [25]:
import os 
import tensorflow as tf
import IPython.display as display
from pathlib import Path
from PIL import Image
import preprocessing as prep

In [3]:
import preprocessing as prep

In [17]:
input_dir = "/Users/richardwang/Documents/Hydroponics/original_dataset/"
data_dir = Path(input_dir)


In [ ]:
# Change the format of all the images in the input directory to the specified format. Can be used on single images or a directory of images.

prep.change_format(input_dir, "png", keep= False)

In [26]:
# Creating a dataset with a pre-defined pipeline
data_original = tf.keras.utils.image_dataset_from_directory(input_dir) # not all image formats supported

Found 281 files belonging to 4 classes.


In [8]:
data_classes = data_original.class_names
data_classes

['deficiency_nitrogen',
 'deficiency_phosphorus',
 'deficiency_potassium',
 'healthy']

In [ ]:
# get a batch of images


healthy = list(data_dir.glob('healthy/*'))
nitro = list(data_dir.glob('deficiency_nitrogen/*'))
phos = list(data_dir.glob('deficiency_phosphorus/*'))
pota = list(data_dir.glob('deficiency_potassium/*'))

for image in healthy[-5:]:
    display.display(Image.open(str(image)))


In [53]:

import cv2
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

from tensorflow.keras.preprocessing.image import ImageDataGenerator


import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

import tensorflow as tf



In [61]:
 #* in this section we can see how to use the ImageDataGenerator to augment the images. Use it as playglround

datagen= ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    shear_range = 0.2,
    zoom_range = 0.2,
    rescale= 1./255,
    fill_mode = 'constant', cval = 255
)
 #* we define the (single) imagefile we want to augment
imagefile = ('/Users/richardwang/Documents/Hydroponics/original_dataset/deficiency_nitrogen/hydroponics_lettuce_P   _01.png')
#we read the imagefile with opencv
x = cv2.imread(imagefile)

#we reshape the image to (1, 256, 256, 3) to fit the input of the model
x = x.reshape((1,) + x.shape)
i = 0
for batch in datagen.flow(x, batch_size=16,
                          save_to_dir='/Users/richardwang/Documents/Hydroponics/original_dataset/', save_prefix='aug', save_format='png'):
    i += 1
    if i > 20:
        break


In [49]:


# define batch and image size here, because it is used in multiple places
batch_size = 32
image_size = (450, 450)

data = tf.keras.utils.image_dataset_from_directory(
    input_dir,
    batch_size= batch_size,
    image_size = image_size,
    shuffle=True,
    seed=42,
    validation_split=0.2, # 20% of the data will automatically be used for validation
    subset="training")



# Print the class names
class_names = data.class_names
print("Class names:", class_names)



# Convert labels to NumPy arrays
X_train = np.concatenate([x for x, _ in data], axis=0)
y_train = np.concatenate([y for _, y in data], axis=0)



# Convert labels to one-hot encoded format
y_train = to_categorical(y_train, num_classes=4)


augmented= ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    shear_range = 0.2,
    zoom_range = 0.2,
    rescale= 1./255,
    fill_mode = 'constant', cval = 20
)

# Fit the augmentation method to the data
augmented.fit(X_train)

s1, s2 = image_size
# Define the baseline model architecture
model = tf.keras.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=( s1, s2, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()


# Train the model
model.fit(augmented.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=len(X_train) / batch_size, epochs=40)


val_data = tf.keras.utils.image_dataset_from_directory(
    input_dir,
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="validation"
)



# Convert labels to NumPy arrays
X_val = np.concatenate([x for x, _ in val_data], axis=0)
y_val = np.concatenate([y for _, y in val_data], axis=0)


# Convert labels to one-hot encoded format
y_val = to_categorical(y_val, num_classes=4)


# Evaluate the model on the validation data
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)





Found 281 files belonging to 4 classes.
Using 225 files for training.
Class names: ['deficiency_nitrogen', 'deficiency_phosphorus', 'deficiency_potassium', 'healthy']
Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_45 (Conv2D)          (None, 448, 448, 32)      896       
                                                                 
 max_pooling2d_45 (MaxPooli  (None, 224, 224, 32)      0         
 ng2D)                                                           
                                                                 
 conv2d_46 (Conv2D)          (None, 222, 222, 64)      18496     
                                                                 
 max_pooling2d_46 (MaxPooli  (None, 111, 111, 64)      0         
 ng2D)                                                           
                                                                 
 conv2d_47 (Conv2D